In [1]:
import numpy as np
from pathlib import Path
from PIL import Image
from torch.utils.data import DataLoader, random_split
import torch
from torch import optim
import torch.nn as nn
from tqdm import tqdm
import torch.nn.functional as F
import logging
from evaluate import evaluate
import os
import albumentations as A
from albumentations.pytorch import ToTensorV2
from collections import OrderedDict

#from model.unet.unet_model import UNet
#from model.segnet.segnet_model import SegNet
#from torchvision.models.segmentation import deeplabv3_resnet101 as DeepLabv3
from model.ensemblenet_model import EnsembleNet


from utils.dice_score import dice_loss
from utils.data_load import KittiDataset
from torchsummaryX import summary

In [2]:
Val_Percent = 0.3
Scale_Percent = 1.0
Batch_Size = 8
learning_rate = 0.001
Pin_Memory = False
epochs = 10

#Image_Size = [384, 1242]
Image_Size = [384, 1216]
#Image_Size = [384,384]
Gradient_Clipping = 1.0

#Num_Class = 31
#Num_Class = 21
Num_Class = 2
Num_Channel = 3
amp = True

#Img_Path =  'etc/training/image_2'
#Mask_Path = 'etc/training/semantic'


#Img_Path =  'data/train/images'
#Mask_Path = 'data/train/labels'

Img_Path =  'data/data_road/training/image_2'
Mask_Path =  'data/data_road/training/semantic'

'''
Img_Path_train =  'etc/train/images'
Mask_Path_train = 'etc/train/masks'
Img_Path_val =  'etc/validation/images'
Mask_Path_val = 'etc/validation/masks'

Img_Path_train =  'data/train/images'
Mask_Path_train = 'data/train/labels'
Img_Path_val =  'data/validation/images'
Mask_Path_val = 'data/validation/labels'
'''

save_checkpoint = False
checkpoint_dir = '../trained'
batch_size = Batch_Size

In [3]:

dirImg = Path(Img_Path)
dirMask = Path(Mask_Path)


'''
dirImg_train = Path(Img_Path_train)
dirMask_train = Path(Mask_Path_train)
dirImg_val = Path(Img_Path_val)
dirMask_val = Path(Mask_Path_val)
'''
dir_checkpoint = Path(checkpoint_dir)

In [4]:
train_transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        #A.VerticalFlip(p=0.5),
        A.Rotate(limit=30, p=0.5),
        #A.RandomBrightnessContrast(p=0.5),
        #A.RandomGamma(p=0.5),
        #A.RandomSnow(p=0.5),
        #A.RandomRain(p=0.5),
        #A.RandomFog(p=0.5),
        #A.RandomSunFlare(p=0.5),
        A.RandomShadow(p=0.5),
        #A.RandomToneCurve(p=0.5),
        #A.GaussNoise(p=0.5),
        #A.Emboss(p=0.5),  # IAAEmboss 대신 Emboss 사용
        #A.Perspective(p=0.5),  # IAAPerspective 대신 Perspective 사용
        #A.CLAHE(p=0.5)
])

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
datasets =  KittiDataset(dirImg, dirMask, Image_Size, Scale_Percent)
#datasets =  KittiDataset(dirImg, dirMask, Image_Size, Scale_Percent, train_transform)
n_val = int(len(datasets) * Val_Percent)
n_train = len(datasets) - n_val
train_set, val_set = random_split(datasets, [n_train, n_val], generator=torch.Generator().manual_seed(0))

loader_args = dict(batch_size=Batch_Size, num_workers= os.cpu_count(), pin_memory=Pin_Memory)
train_loader = DataLoader(train_set, shuffle=True, drop_last = True, **loader_args)
val_loader = DataLoader(val_set, shuffle=False, drop_last=True, **loader_args)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 289/289 [00:00<00:00, 728.16it/s]


dirImg_train = Path(Img_Path_train)
dirMask_train = Path(Mask_Path_train)
dirImg_val = Path(Img_Path_val)
dirMask_val = Path(Mask_Path_val)

dir_checkpoint = Path(checkpoint_dir)

#train_datasets =  KittiDataset(dirImg_train, dirMask_train, Image_Size, 'train', Scale_Percent, transform = train_transform)
train_datasets =  KittiDataset(dirImg_train, dirMask_train, Image_Size, 'train', Scale_Percent)
val_datasets =  KittiDataset(dirImg_val, dirMask_val, Image_Size, 'validation', Scale_Percent)
n_train = len(train_datasets)

loader_args = dict(batch_size=Batch_Size, num_workers= os.cpu_count(), pin_memory=Pin_Memory)
train_loader = DataLoader(train_datasets, shuffle=True, drop_last = True, **loader_args)
val_loader = DataLoader(val_datasets, shuffle=False, drop_last=True, **loader_args)

#datasets =  KittiDataset(dirImg, dirMask, Image_Size, Scale_Percent)
#datasets =  KittiDataset(dirImg, dirMask, Image_Size, Scale_Percent, train_transform)
n_val = int(len(datasets) * Val_Percent)
n_train = len(datasets) - n_val
train_set, val_set = random_split(datasets, [n_train, n_val], generator=torch.Generator().manual_seed(0))

loader_args = dict(batch_size=Batch_Size, num_workers= os.cpu_count(), pin_memory=Pin_Memory)
train_loader = DataLoader(train_set, shuffle=True, drop_last = True, collate_fn = cus_collate_fn, **loader_args)
val_loader = DataLoader(val_set, shuffle=False, drop_last=True, **loader_args)

In [7]:
model = EnsembleNet('segnet', Num_Channel, Num_Class)
model = model.to(memory_format=torch.channels_last, device = device)

summary(model, torch.zeros(2,3,384,1242).to(device))

In [8]:
# deeplab model output shape torch.Size([4, 31, 38, 124])

In [8]:
# 4. Set up the optimizer, the loss, the learning rate scheduler and the loss scaling for AMP
optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=1e-8, momentum=0.9)
#optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-8)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2)  # goal: maximize Dice score
grad_scaler = torch.cuda.amp.GradScaler(enabled=amp)
criterion = nn.CrossEntropyLoss()
global_step = 0

In [9]:
# 5. Begin training
for epoch in range(1, epochs + 1):
    model.train()
    epoch_loss = 0
    with tqdm(total=n_train, desc=f'Epoch {epoch}/{epochs}', unit='img') as pbar:
        for batch in train_loader:
            images, true_masks = batch['image'], batch['mask']

            images = images.to(device=device, dtype=torch.float32, memory_format=torch.channels_last)
            true_masks = true_masks.to(device=device, dtype=torch.long)

            with torch.autocast(device.type if device.type != 'mps' else 'cpu', enabled=amp):
                masks_pred = model(images)
                if isinstance(masks_pred, OrderedDict):
                    masks_pred = masks_pred['out']
                             
                try:
                    mn_cls = model.n_classes
                except:
                    mn_cls = model.classifier[-1].out_channels
                    
                
                if mn_cls == 1:
                    loss = criterion(masks_pred.squeeze(1), true_masks.float())
                    loss += dice_loss(F.sigmoid(masks_pred.squeeze(1)), true_masks.float(), multiclass=False)
                else:
                    loss = criterion(masks_pred, true_masks)
                    loss += dice_loss(
                        F.softmax(masks_pred, dim=1).float(),
                        F.one_hot(true_masks, mn_cls).permute(0, 3, 1, 2).float(),
                        multiclass=True
                    )
                

            optimizer.zero_grad(set_to_none=True)
            grad_scaler.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), Gradient_Clipping)
            grad_scaler.step(optimizer)
            grad_scaler.update()

            pbar.update(images.shape[0])
            global_step += 1
            epoch_loss += loss.item()


            # Evaluation round
            division_step = (n_train // (5 * batch_size))
            if division_step > 0:
                if global_step % division_step == 0:

                    val_score = evaluate(model, val_loader, device, amp)
                    
                    scheduler.step(val_score)

                    #logging.info('Validation Dice score: {}'.format(val_score))
                    print('Training Dice Loss: {}'.format(loss))
                    print('Validation Dice score: {}'.format(val_score))
                                
            
    if save_checkpoint:
        Path(dir_checkpoint).mkdir(parents=True, exist_ok=True)
        torch.save(model.state_dict(), str(dir_checkpoint / 'checkpoint_epoch{}.pth'.format(epoch + 1)))

Epoch 1/10:  24%|███████████████████████████████████████████████████████▌                                                                                                                                                                                   | 48/203 [00:07<00:22,  6.91img/s]

Training Dice Loss: 0.7869431972503662
Validation Dice score: 0.002651951741427183


Epoch 1/10:  43%|█████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                                                                                                     | 88/203 [00:10<00:14,  7.96img/s]

Training Dice Loss: 0.6706660389900208
Validation Dice score: 1.367285885206515e-11


Epoch 1/10:  63%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                                                                      | 128/203 [00:13<00:09,  7.83img/s]

...

Training Dice Loss: 0.26844117045402527
Validation Dice score: 0.8184521794319153


Epoch 10/10:  83%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                        | 168/203 [00:14<00:04,  8.10img/s]

Training Dice Loss: 0.1621471643447876
Validation Dice score: 0.8298660516738892


Epoch 10/10:  99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌   | 200/203 [00:17<00:00, 11.50img/s]

Training Dice Loss: 0.18551212549209595
Validation Dice score: 0.8257239460945129
